In [50]:
#data: https://datahub.io/core/s-and-p-500-companies#python
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
from datapackage import Package
import json
import random

In [51]:
package = Package('https://datahub.io/core/s-and-p-500-companies/datapackage.json')
# print processed tabular data (if exists any)
for resource in package.resources:
    if resource.descriptor['datahub']['type'] == 'derived/csv':
        ticker_list = resource.read()

In [25]:
#return list of world indices symbols
df_list = pd.read_html('https://finance.yahoo.com/world-indices/')
indices = df_list[0]
#for asset in asset_list:
asset_list = []
for i in range(0, 17):
    asset_dict = {}  # Initialize a new dictionary for each asset
    asset_dict['yfticker'] = indices.iloc[i, 0]
    asset_dict['name'] = indices.iloc[i, 1]
    asset_list.append(asset_dict)  # Append the dictionary to the list

In [26]:
# There are 4 tables on the Wikipedia page
# we want the last table
payload=pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')
dfnsd = payload[4]
#spx500 tickers
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv('S&P500-Info.csv')
dfspx = df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])

In [27]:
#spx500 tickers
#table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
file_path = 'S&P500-Info.csv'
df = pd.read_csv(file_path)
#df.to_csv('S&P500-Info.csv')
#df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])

In [ ]:
def extract_random_tickers(data, num_tickers=5):
    # Ensure there are enough lists in the data
    if len(data) < num_tickers:
        print(f"Warning: There are less than {num_tickers} lists in the data.")
        num_tickers = len(data)
    
    # Randomly sample 'num_tickers' from the data
    sampled_lists = random.sample(data, num_tickers)
    
    # Extract tickers from the sampled lists and format as dictionaries
    random_tickers = [{'yfticker': ticker_list[0]} for ticker_list in sampled_lists]
    
    return random_tickers

In [29]:
def return_sector_symbols(df, sector):
    # Filter rows where 'GICS Sector' is equal to the specified sector
    sector_symbols = df[df['GICS Sector'] == sector][['Symbol']]
    # Convert the Pandas DataFrame to a list of dictionaries
    symbol_list = [{'yfticker': symbol} for symbol in sector_symbols['Symbol'].tolist()]
    return symbol_list

In [ ]:
'''
# Count frequencies of unique values in the 'GICS Sector' column
sector_counts = df['GICS Sector'].value_counts()
# Display the result
print(sector_counts)
#indices = return_sector_symbols(df,'Real Estate')
#indices = return_random_symbols( df[df['GICS Sector'] == 'Industrials'], num_symbols=5)
#indices
'''

In [30]:
def return_random_symbols(df, num_symbols=5):
    # Ensure there are enough symbols in the DataFrame
    if len(df) < num_symbols:
        print(f"Warning: There are less than {num_symbols} symbols in the DataFrame.")
        num_symbols = len(df)
    # Randomly sample 'num_symbols' from the DataFrame
    sampled_symbols = df['Symbol'].sample(n=num_symbols, replace=False).tolist()
    # Convert the sampled symbols to a list of dictionaries
    symbol_list = [{'yfticker': symbol} for symbol in sampled_symbols]
    return symbol_list

In [31]:
def print_min_max_dates(list_of_stock_data):
    #used to find out inconsistencies in stock datas
    for i, stock_data in enumerate(list_of_stock_data):
        df = list_of_stock_data[i]  # Replace 'dataframe' with the actual key holding the DataFrame
        if 'date' in df.columns:
            min_date = df.iloc[:, 0].min()
            max_date = df.iloc[:, 0].max()

            print(f"DataFrame {i + 1}:")
            print(f"  Minimum Date: {min_date}")
            print(f"  Maximum Date: {max_date}")
        else:
            print(f"DataFrame {i + 1}: Missing 'date' column.")

In [32]:
def make_dataframes_same_time(list_of_dataframes):
    # Find the common time range
    common_start_date = max(df.iloc[:, 0].min() for df in list_of_dataframes)
    common_end_date = min(df.iloc[:, 0].max() for df in list_of_dataframes)

    # Trim each DataFrame to the common time range
    trimmed_dataframes = []
    for df in list_of_dataframes:
        # Trim the DataFrame to the common time range
        df_trimmed = df[(df.iloc[:, 0] >= common_start_date) & (df.iloc[:, 0] <= common_end_date)]
        df_trimmed = df_trimmed.reset_index(drop=True)
        trimmed_dataframes.append(df_trimmed)

    return trimmed_dataframes